In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType, DoubleType
from pyspark.sql.functions import col, to_timestamp

# Initialize SparkSession
spark = SparkSession.builder.appName("CarwashReviews").getOrCreate()

# Define the schema
schema = StructType([
    StructField("random1", StringType(), True),        
    StructField("random2", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("random4", StringType(), True),
    StructField("random5", StringType(), True),
    StructField("label", DoubleType(), False),
    StructField("reviews", StringType(), True),
    StructField("random8", StringType(), True),
    StructField("random9", StringType(), True),
])

 

# # Create DataFrame
# df = spark.createDataFrame(data, schema)

# df = df.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))


# # Show the DataFrame
# df.show(truncate=False)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 20:22:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, lit, length
from pyspark.sql.types import StructType, StructField, StringType
from datetime import datetime
import pandas as pd
from google.cloud import storage
from pyspark.sql import functions as F
from pyspark.ml.feature import Imputer
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, TimestampType, StringType
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Tokenizer, VectorAssembler
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql.functions import from_csv


logs = []

schema = StructType([
    StructField("random1", StringType(), True),        
    StructField("random2", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("random4", StringType(), True),
    StructField("random5", StringType(), True),
    StructField("label", DoubleType(), False),
    StructField("reviews", StringType(), True),
    StructField("random8", StringType(), True),
    StructField("random9", StringType(), True),
])

def ml_task(df):
    # df = spark.createDataFrame(df, schema=schema)

    # Convert timestamp to numeric (e.g., epoch time)
    df = df.withColumn("timestamp_numeric", F.col("timestamp").cast("long"))

    # Perform your operations on the numeric column
    # Example: fill missing values with a constant value or mode
    df = df.na.fill({"timestamp_numeric": 0})

    # Convert back to timestamp
    df = df.withColumn("timestamp", F.col("timestamp_numeric").cast("timestamp"))

    # Convert timestamp to numeric (e.g., epoch time)
    df = df.withColumn("timestamp_numeric", F.col("timestamp").cast("long"))

#     # Initialize the Imputer
#     imputer = Imputer(inputCols=["timestamp_numeric"], outputCols=["timestamp_numeric"])

#     # Fit the Imputer model and transform the DataFrame
#     imputed_df = imputer.fit(df).transform(df)

#     # Convert back to timestamp
#     imputed_df = imputed_df.withColumn("timestamp", F.col("timestamp_numeric").cast("timestamp"))

    # Fill null values in 'label' with a default value (e.g., 0.0)
    df = df.na.fill({"label": 0.0})

    # Fill null values in 'label' with a default value (e.g., 0.0)
    df = df.na.fill({"reviews": ''})

    # Path where the pipeline model is saved
    pipeline_model_path = "gs://pyq/pipeline_model"

    # Load the pipeline model
    loaded_pipeline_model = PipelineModel.load(pipeline_model_path)

    # Check the loaded model
    print(loaded_pipeline_model)

    transformed_data = loaded_pipeline_model.transform(df)

    # Assemble features
    assembler = VectorAssembler(inputCols=["onehot_features", "timestamp_numeric"], outputCol="features")
    assembled_df = assembler.transform(transformed_data)
    
    model_path = "gs://pyq/lrmodel"
    lr_model = LogisticRegressionModel.load(model_path)
    
    prediction = lr_model.transform(assembled_df)
    print("\n\n"*10) 
    prediction.show()
    print("\n\n"*10)
    null_counts = assembled_df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in assembled_df.columns])

    # Show the result
    null_counts.show()

In [3]:
myTable = spark.read.format("csv").schema(schema).load("test_part-00039-808f9971-b2b6-4a6f-b8cf-0822a68f365f-c000.csv")


In [4]:
ml_task(myTable)

Py4JJavaError: An error occurred while calling o58.partitions.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:275)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$p

24/08/24 20:22:20 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
